In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import cos, asin, sqrt, pi #calculate distance

# Preprocessing Tasks

### Loading all datasets

In [ ]:
cases_train = pd.read_csv('./datasets/cases_2021_train.csv')
cases_test = pd.read_csv('./datasets/cases_2021_test.csv')
cases_location = pd.read_csv('./datasets/location_2021.csv')

## 1.1 Cleaning messy outcome labels
Datasets involved: cases_2021_train.csv

In [ ]:
cases_train.groupby('outcome').size()

In [ ]:
# labels_test = {
#     'hospitalized': {'Discharged', 'Discharged from hospital', 'Hospitalized', 'critical condition','discharge', 'discharged'},
#     'nonhospitalized': {'Alive', 'Receiving Treatment', 'Stable', 'Under treatment', 'recovering at home 03.03.2020', 'released from quarantine', 'stable', 'stable condition'},
#     'deceased': {'Dead', 'Death', 'Deceased', 'Died', 'death', 'died'},
#     'recovered': {'Recovered', 'recovered'}
# }

### Mapping similar outcomes

In [ ]:
labels = {
    'Discharged': 'hospitalized', 'Discharged from hospital': 'hospitalized', 'Hospitalized': 'hospitalized', 'critical condition': 'hospitalized', 'discharge': 'hospitalized', 'discharged': 'hospitalized', 
    'Alive': 'nonhospitalized', 'Receiving Treatment': 'nonhospitalized', 'Stable': 'nonhospitalized', 'Under treatment': 'nonhospitalized', 'recovering at home 03.03.2020': 'nonhospitalized', 'released from quarantine': 'nonhospitalized', 'stable': 'nonhospitalized', 'stable condition': 'nonhospitalized', 
    'Dead': 'deceased', 'Death': 'deceased', 'Deceased': 'deceased', 'Died': 'deceased', 'death': 'deceased', 'died': 'deceased',
     'Recovered': 'recovered', 'recovered': 'recovered'
}

In [ ]:
cases_train['outcome_group'] = cases_train['outcome'].map(labels)

In [ ]:
cases_train

In [ ]:
cases_train = cases_train.drop(columns=['outcome'])

In [ ]:
cases_train

In [ ]:
cases_train.groupby('outcome_group').size()

## 1.3 Exploratory Data Analysis 

Ideas to explore

Probably want to convert date_information to a datetime object 

Categorical Attributes: Province, Country, Chronic disease, outcome_group, Sex
Numerical: Age, Date
By: country, Age, Sex
Age range: 18 - 30, < 18, 60 >


### 1.3.1 train / test dataset

In [ ]:
stats_cases_train = cases_train.describe(include='all')
stats_cases_train = stats_cases_train.append(pd.DataFrame([cases_train.isna().sum()], index=['null'])) # add count of null values
stats_cases_train = stats_cases_train.iloc[[0,11,1]] # keep [count, null, unique]
stats_cases_train

In [ ]:
stats_cases_test = cases_test.describe(include='all').round(2)
stats_cases_test = stats_cases_test.append(pd.DataFrame([cases_test.isna().sum()], index=['null'])) # add count of null values
stats_cases_test = stats_cases_test.iloc[[0,11,1]] # keep [count, null, unique]
stats_cases_test

In [ ]:
# age attribute
plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.hist(pd.to_numeric(cases_train['age'], errors='coerce'), bins=10)
plt.title('Train Cases by Age')
plt.xlabel('Age')
plt.ylabel('Count')

plt.subplot(1,2,2)
plt.hist(pd.to_numeric(cases_test['age'], errors='coerce'), bins=10)
plt.title('Test Cases by Age')
plt.xlabel('Age')
plt.ylabel('Count')
#plt.show()
plt.savefig('plots/task-1.3/Train_Test_Age.svg')

In [ ]:
# sex attribute
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plt.bar(cases_train['sex'].value_counts().index,cases_train['sex'].value_counts())
plt.title('Train Cases by Sex')
plt.ylabel('Count')

plt.subplot(1,2,2)
plt.bar(cases_test['sex'].value_counts().index,cases_test['sex'].value_counts())
plt.title('Test Cases by Sex')
plt.ylabel('Count')
plt.tight_layout()
#plt.show()
plt.savefig('plots/task-1.3/Train_Test_Sex.svg')

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
values = cases_train[['age', 'sex']]
plt.hist(pd.to_numeric(values.loc[values['sex']=='male', 'age'], errors='coerce'), bins=10)
plt.hist(pd.to_numeric(values.loc[values['sex']=='female', 'age'], errors='coerce'), bins=10)
plt.title('Train Cases by Age and Sex')
plt.legend(['male', 'female'])
plt.xlabel('Age')
plt.ylabel('Count')

plt.subplot(1,2,2)
values = cases_test[['age', 'sex']]
plt.hist(pd.to_numeric(values.loc[values['sex']=='male', 'age'], errors='coerce'), bins=10)
plt.hist(pd.to_numeric(values.loc[values['sex']=='female', 'age'], errors='coerce'), bins=10)
plt.title('Test Cases by Age and Sex')
plt.legend(['male', 'female'])
plt.xlabel('Age')
plt.ylabel('Count')
#plt.show()
plt.savefig('plots/task-1.3/Train_Test_Age_Sex.svg')

In [ ]:
# country attribute
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
values = pd.DataFrame(cases_train['country'].value_counts()).reset_index()
values['index'] = values.apply(lambda x: x['index'] if x.country>100 else 'Other', axis=1)# sumarize others
plt.bar(values['index'], values['country'])
plt.title('Train Cases by Country')
plt.ylabel('Count')

plt.subplot(1,2,2)
values = pd.DataFrame(cases_test['country'].value_counts()).reset_index()
values['index'] = values.apply(lambda x: x['index'] if x.country>60 else 'Other', axis=1)# sumarize others
plt.bar(values['index'], values['country'])
plt.title('Test Cases by Country')
plt.ylabel('Count')
plt.tight_layout()
#plt.show()
plt.savefig('plots/task-1.3/Train_Test_Country.svg')

In [ ]:
# country attribute
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
values = pd.DataFrame(cases_train['country'].value_counts()).reset_index()
values = values.drop(index=[0, 1])
values['index'] = values.apply(lambda x: x['index'] if x.country>60 else 'Other', axis=1)# sumarize others
plt.bar(values['index'], values['country'])
plt.title('Train Cases by Country (w/o India, Philippines)')
plt.ylabel('Count')

plt.subplot(1,2,2)
values = pd.DataFrame(cases_test['country'].value_counts()).reset_index()
values = values.drop(index=[0, 1])
values['index'] = values.apply(lambda x: x['index'] if x.country>30 else 'Other', axis=1)# sumarize others
plt.bar(values['index'], values['country'])
plt.title('Test Cases by Country (w/o India, Philippines)')
plt.ylabel('Count')
#plt.show()
plt.savefig('plots/task-1.3/Train_Test_Country2.svg')

In [ ]:
# province attribute - India
plt.figure(figsize=(20,6))
plt.subplot(1,2,1)
values = cases_train[cases_train['country']=='India']
values = pd.DataFrame(values['province'].value_counts().sort_values(ascending=True)).reset_index()
values['index'] = values.apply(lambda x: x['index'] if x.province>500 else 'Other', axis=1)#sumarize others
plt.barh(values['index'], values['province'])
plt.title('Train Cases by Province for India')
plt.xlabel('Count')

plt.subplot(1,2,2)
values = cases_test[cases_test['country']=='India']
values = pd.DataFrame(values['province'].value_counts().sort_values(ascending=True)).reset_index()
values['index'] = values.apply(lambda x: x['index'] if x.province>500 else 'Other', axis=1)#sumarize others
plt.barh(values['index'], values['province'])
plt.title('Test Cases by Province for India')
plt.xlabel('Count')
plt.tight_layout()

#plt.show()
plt.savefig('plots/task-1.3/Train_Test_Provinces_India.svg')

In [ ]:
# province attribute - Philippines
plt.figure(figsize=(20,6))
plt.subplot(1,2,1)
values = cases_train[cases_train['country']=='Philippines']
values = pd.DataFrame(values['province'].value_counts().sort_values(ascending=True)).reset_index()
values['index'] = values.apply(lambda x: x['index'] if x.province>10 else 'Other', axis=1)#sumarize others
plt.barh(values['index'], values['province'])
plt.title('Train Cases by Province for Philippines')
plt.xlabel('Count')

plt.subplot(1,2,2)
values = cases_test[cases_test['country']=='Philippines']
values = pd.DataFrame(values['province'].value_counts().sort_values(ascending=True)).reset_index()
values['index'] = values.apply(lambda x: x['index'] if x.province>10 else 'Other', axis=1)#sumarize others
plt.barh(values['index'], values['province'])
plt.title('Test Cases by Province for Philippines')
plt.xlabel('Count')
plt.tight_layout()

#plt.show()
plt.savefig('plots/task-1.3/Train_Test_Provinces_Philippines.svg')

In [ ]:
# latitude / longitude attribute
plt.figure(figsize=(16,5))
plt.subplot(1,2,1)
plt.plot(cases_train['latitude'], cases_train['longitude'], '.')
plt.title('Train Cases by Latitude / Longitude')
plt.xlabel('Latitude')
plt.ylabel('Longitude')

plt.subplot(1,2,2)
plt.plot(cases_test['latitude'], cases_test['longitude'], '.')
plt.title('Test Cases by Latitude / Longitude')
plt.xlabel('Latitude')
plt.ylabel('Longitude')

#plt.show()
plt.savefig('plots/task-1.3/Train_Test_Lat_Lon.svg')

In [ ]:
# date_confirmation attribute

# add month attribute
cases_train['month'] = cases_train.apply(lambda x: str(x.date_confirmation)[3:5] + str(x.date_confirmation)[6:10], axis=1)
cases_train = cases_train.replace("", np.NaN)
cases_test['month'] = cases_test.apply(lambda x: str(x.date_confirmation)[3:5] + str(x.date_confirmation)[6:10], axis=1)
cases_test = cases_test.replace("", np.NaN)

plt.figure(figsize=(16, 5))
plt.subplot(1,2,1)
plt.bar(cases_train['month'].value_counts().index, cases_train['month'].value_counts())
plt.title('Train Cases by Confirmation Date')
plt.xlabel('Month')
plt.ylabel('Count')

plt.subplot(1,2,2)
plt.bar(cases_test['month'].value_counts().index, cases_test['month'].value_counts())
plt.title('Test Cases by Confirmation Date')
plt.xlabel('Month')
plt.ylabel('Count')

#plt.show()
plt.savefig('plots/task-1.3/Train_Test_Date.svg')

In [ ]:
# chronic disease  attribute
plt.figure(figsize=(9,4))
plt.subplot(1,2,1)
plt.bar(['True','False'],cases_train['chronic_disease_binary'].value_counts())
plt.title('Train Cases by Chronic Disease')
plt.ylabel('Count')

plt.subplot(1,2,2)
plt.bar(['True','False'],cases_test['chronic_disease_binary'].value_counts())
plt.title('Test Cases by Chronic Disease')
plt.ylabel('Count')
plt.tight_layout()

#plt.show()
plt.savefig('plots/task-1.3/Train_Test_Chronic_Disease.svg')

In [ ]:
#outcome group attribute
plt.bar(cases_train['outcome_group'].value_counts().index,cases_train['outcome_group'].value_counts())
plt.title('Train Cases by Outcome Group')
plt.ylabel('Count')
#plt.show()
plt.savefig('plots/task-1.3/Train_Test_Outcome_Group.svg')

### 1.3.2 location dataset

In [ ]:
stats_cases_location = cases_location.describe(include="all").round(2)
stats_cases_location = stats_cases_location.append(pd.DataFrame([cases_location.isna().sum()], index=['null'])) # add count of null values
stats_cases_location = stats_cases_location.iloc[[0,11,1, 4, 5, 6, 10]] # keep [count, null, unique]
stats_cases_location

In [ ]:
# country / region attribute
plt.figure(figsize=(16,5))
values = pd.DataFrame(cases_location['Country_Region'].value_counts()).reset_index()
values['index'] = values.apply(lambda x: x['index'] if x['Country_Region']>15 else 'Other', axis=1)# sumarize others
plt.bar(values['index'], values['Country_Region'])
plt.title('Data Points per Country / Region')
plt.ylabel('Count')
plt.xticks(rotation=45)

#plt.show()
plt.savefig('plots/task-1.3/Location_Region.svg')

In [ ]:
# latitude / longitude attribute
plt.figure(figsize=(10,7))
plt.plot(cases_location['Lat'], cases_location['Long_'], '.')
plt.title('Data Points by Latitude / Longitude')
plt.xlabel('Latitude')
plt.ylabel('Longitude')

#plt.show()
plt.savefig('plots/task-1.3/Location_Lat_Lon.svg')

In [ ]:
# confirmed, recovered, active, deaths attribute
values = cases_location.groupby('Country_Region').sum()
values = values.nlargest(25, 'Confirmed') #get Top 25 countries

plt.figure(figsize=(16,5))
plt.bar(values.index, values['Recovered'], color='green')
plt.bar(values.index, values['Active'], color='orange')
plt.bar(values.index, values['Deaths'],  color='black')
plt.xticks(rotation=45)
plt.legend(['Recovered', 'Active', 'Deaths'])
plt.title('Confirmed Cases by Country')
plt.ylabel('Count')
plt.ticklabel_format(axis='y', style='plain') # prevent scientific notation

#plt.show()
plt.savefig('plots/task-1.3/Location_Confirmed.svg')

In [ ]:
# incident rate attribute
values = cases_location.groupby('Country_Region').mean()
values = values.nlargest(25, 'Incident_Rate')

plt.figure(figsize=(16,5))
plt.plot(values.index, values['Incident_Rate'])
plt.xticks(rotation=45)
plt.title('Incident Rate by Country')
plt.ylabel('Incident Rate')

#plt.show()
plt.savefig('plots/task-1.3/Location_Incident_Rate.svg')

In [ ]:
# case_fatality attribute
values = cases_location.groupby('Country_Region').mean()
values = values.nlargest(25, 'Case_Fatality_Ratio')

plt.figure(figsize=(16,5))
plt.plot(values.index, values['Case_Fatality_Ratio'])
plt.xticks(rotation=45)
plt.title('Case Fatality Ratio by Country')
plt.ylabel('Fatality Rate')

#plt.show()
plt.savefig('plots/task-1.3/Location_Fatality_Rate.svg')

## 1.4 Data Cleaning and Imputing Missing Values
Datasets involved: cases_2021_train.csv, cases_2021_test.csv, location_2021.csv

In [ ]:
# Primary: age column -> remove all entries that are NaN
# format age to standard integer
# potential approaches: 

# other columns -> impute, what strategies can we use to impute missing values of different columns
# additional information: best action is replace NaN with ""

In [ ]:
# Removing NaN's from age column in train and test datasets

cases_train = cases_train[cases_train['age'].notna()].reset_index()
cases_test = cases_test[cases_test['age'].notna()].reset_index()


In [ ]:
# Strip all whitespace from 'age' columns
cases_train['age'] = cases_train['age'].str.strip()
cases_test['age'] = cases_test['age'].str.strip()

In [ ]:
# age range greater than 10 should be removed?

# FIX: Remove empty spaces from 'age' column
# FOUND EDGE CASE: ['80', ''] -> 80 - 

def range_to_num(age):
    range = 10

    a_list = age.split('-')
    # print(a_list)

    if (a_list[0] == ''):
        return round(float(a_list[1]))
    elif ((len(a_list)) == 2 and (a_list[1] == '')):
        return round(float(a_list[0]))

    map_ints = map(float, a_list)
    map_list = list(map_ints)

    if (len(map_list) == 1):
        return round(map_list[0])  

    if ((map_list[1] - map_list[0]) > range):
        return 'remove'
    else:
        return round((map_list[1] + map_list[0]) / 2)


In [ ]:
cases_train = cases_train.drop(columns=['index'])
cases_test = cases_test.drop(columns=['index'])

In [ ]:
# STEPS TO CLEAN VALUES IN AGE COLUMN
#1. Format all values with 'x-x' range to a single int value or 'remove'
#2. Remove all entries labelled 'remove' 
#3. Convert all column entries to int
#4. Use round() on all values to get rid of 0.3, 0.5, 0.8 values -> some babies

In [ ]:
# Cleaning train dataset - age

cases_train['age'] = cases_train['age'].apply(lambda x: range_to_num(x))
cases_train = cases_train[cases_train['age'] != 'remove']
cases_train['age'] = pd.to_numeric(cases_train['age'])
cases_train['age'] = cases_train['age'].apply(lambda x: round(x))

In [ ]:
# sex	province	country	latitude	longitude	date_confirmation	additional_information	source	chronic_disease_binary	outcome_group
# sex: NaN convert to 'unknown'
# province: can use latitude and longitude to get province -> need to find appropriate dataset / function to determine geographical location
# country: has no NaN values -> is okay
# date_confirmation: some NaN values -> think its okay to keep these entries -> convert NaN's to 'unknown'
# additional_information: some NaN values -> convert to 'unknown'
# source: some NaN values -> convert to 'unknown' ** might want to remove sources that are unknown
# chronic_disease_binary: no NaN values
# outcome_group: no NaN values

In [ ]:
cases_train[['sex', 'date_confirmation', 'additional_information', 'source']] = cases_train[['sex', 'date_confirmation', 'additional_information', 'source']].fillna('unknown')
cases_test[['sex', 'date_confirmation', 'additional_information', 'source']] = cases_test[['sex', 'date_confirmation', 'additional_information', 'source']].fillna('unknown')

In [ ]:
# Clean the locations data
cases_location.dropna(subset=['Incident_Rate', 'Case_Fatality_Ratio'], inplace=True)

### 1.5 Dealing With Outliers

#### Ideas
- For the attributes in our dataset, not many could be determined outliers
- Age could be an outlier, if the majority of deceased are "older" and there is a few random "young" cases -> wouldn't want to remove these though
- Date_confirmation could be an outlier -> this could occur due to misinput and if the dates are way before Covid-19 was detected throughout the world, can remove the entry
- Source could be an attribute to use and remove entries -> if the entry has no source how can we "trust" that it is truthful/accurate

#### Findings Train / Test
- Age: all values seem reasonable
- Sex: all values seem reasonable
- Province: some countries do not report any provinces
- Country: all values seem reasonable
- latitude / longitude: all values seems reasonable
- date_confirmation: all values seem reasonable
- additional information: does not make sense to check for outliers
- source: does not make sense to check for outliers
- chronic_disease_binary: False even if chronic disease is mentioned
- outcome_group: all values seem reasonable

In [ ]:
# set binary to True if chronic disease is mentioned in addional information
cases_test['chronic_disease_binary'] = cases_test.apply(lambda x: True if pd.notnull(x.additional_information)
                                                        and "chronic" in x.additional_information.lower()
                                                        else x.chronic_disease_binary, axis=1)

cases_train['chronic_disease_binary'] = cases_train.apply(lambda x: True if pd.notnull(x.additional_information)
                                                          and "chronic" in x.additional_information.lower()
                                                          else x.chronic_disease_binary, axis=1)

In [ ]:
# Remove locations that have confirmed cases > 1000 and case_fatality_ratio < 0.1
cases_location = cases_location[~((cases_location['Confirmed'] > 1000) & (cases_location['Case_Fatality_Ratio'] < 0.1))].reset_index(drop=True)

## 1.6 Joining the cases and location dataset

In [ ]:
# Fix country formatting inconsistencies
def fix_country(country):
    if country == 'Korea, South':
        return 'South Korea'
    if country == 'US':
        return 'United States'
    return country

cases_location['Country_Region'] = cases_location['Country_Region'].apply(fix_country)

In [ ]:
# Group the location data by province and country, recalculating the COVID data.
cases_location['Province_State'].fillna('', inplace=True)
cases_location['Population'] = cases_location['Confirmed'] * 100000 / cases_location['Incident_Rate']
grouped_locations = cases_location.groupby(['Province_State', 'Country_Region']).agg({'Lat': 'mean', 
                                                                                      'Long_': 'mean', 
                                                                                      'Confirmed': 'sum',
                                                                                     'Deaths': 'sum',
                                                                                     'Recovered': 'sum',
                                                                                     'Active': 'sum',
                                                                                     'Population': 'sum'}).reset_index()
grouped_locations['Incident_Rate'] = grouped_locations['Confirmed'] / grouped_locations['Population'] * 100000 
grouped_locations['Case_Fatality_Ratio'] = grouped_locations['Deaths'] / grouped_locations['Confirmed'] * 100 

In [ ]:
# calculates distance in kilometres between two points given by latitude and longitude
# function slightly adjusted from:
# https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula/21623206
def calc_distance(lat1, lon1, lat2, lon2):
    earth_radius = 6371 # kilometres    
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 2 * earth_radius * asin(sqrt(a)) #return in km

In [ ]:
# Replace missing countries in the cases data with the closest country in the locations data.
def get_country(lat, lon):
    countries = cases_location[['Country_Region', 'Lat', 'Long_']]
    countries = countries.groupby(['Country_Region']).mean().dropna().reset_index()
    countries['dist'] = countries.apply(lambda x: calc_distance(x.Lat, x.Long_, lat, lon), axis=1)
    
    return countries.loc[countries['dist'].idxmin(axis=0)]['Country_Region']

cases_train['country'] = cases_train.apply(lambda x: get_country(x.latitude, x.longitude) if pd.isnull(x['country']) else x.country, axis=1)
cases_test['country'] = cases_test.apply(lambda x: get_country(x.latitude, x.longitude) if pd.isnull(x['country']) else x.country, axis=1)

In [ ]:
# Replace provinces that are missing or don't match anything in the locations dataset with the closest province from the locations dataset
def get_province(country, lat, lon):
    provinces = cases_location[['Country_Region', 'Province_State', 'Lat', 'Long_']]
    provinces = provinces.groupby(['Country_Region', 'Province_State']).mean().dropna().reset_index()
    provinces = provinces[provinces['Country_Region']==country]
    
    if provinces.empty:
        return np.nan
    
    provinces['dist'] = provinces.apply(lambda x: calc_distance(x.Lat, x.Long_, lat, lon), axis=1)
    
    return provinces.loc[provinces['dist'].idxmin(axis=0)]['Province_State']

location_set = set()

def add_to_location_set(row):
    location_set.add((row['Province_State'], row['Country_Region']))
    
grouped_locations.apply(add_to_location_set, axis=1)
cases_train['province'] = cases_train.apply(lambda x: get_province(x.country, x.latitude, x.longitude) if pd.isnull(x['province']) or (x.province, x.country) not in location_set else x.province, axis=1)
cases_test['province'] = cases_test.apply(lambda x: get_province(x.country, x.latitude, x.longitude) if pd.isnull(x['province']) or (x.province, x.country) not in location_set else x.province, axis=1)

In [ ]:
# Merge the training and test data with the location data
combined_train = pd.merge(cases_train, grouped_locations, how='inner', left_on=['country', 'province'], right_on = ['Country_Region', 'Province_State'])
combined_test = pd.merge(cases_test, grouped_locations, how='inner', left_on=['country', 'province'], right_on = ['Country_Region', 'Province_State'])

combined_train.columns = combined_train.columns.str.lower()
combined_test.columns = combined_train.columns.str.lower()

In [ ]:
# Save processed data to results directory
combined_train.to_csv('results/cases_2021_train_processed.csv', index=False)
combined_test.to_csv('results/cases_2021_test_processed.csv', index=False)
grouped_locations.to_csv('results/location_2021_processed.csv', index=False)

## 1.7 Feature Selection

In [ ]:
# Extract the features from the combined data
train_features = combined_train[['age', 'sex', 'province', 'country', 'date_confirmation', 'chronic_disease_binary', 'confirmed', 'deaths', 'population', 'incident_rate', 'case_fatality_ratio', 'outcome_group']]
test_features = combined_test[['age', 'sex', 'province', 'country', 'date_confirmation', 'chronic_disease_binary', 'confirmed', 'deaths', 'population', 'incident_rate', 'case_fatality_ratio', 'outcome_group']]

In [ ]:
# Save extracted features to results directory
train_features.to_csv('results/cases_2021_train_processed_features.csv', index=False)
test_features.to_csv('results/cases_2021_test_processed_features.csv', index=False)